# Detection

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pickle
import csv
import time
import pandas as pd

with open('Hand.pkl', 'rb') as f:
    model = pickle.load(f)

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

font = cv2.FONT_HERSHEY_SIMPLEX

# cap = cv2.VideoCapture(videoPATH)
cap = cv2.VideoCapture(0)

window_name = "Mediapipe"
cv2.namedWindow(window_name, cv2.WINDOW_FULLSCREEN)

with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
    while True:
        ret, image = cap.read()
        
        image = cv2.flip(image, 1) 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # image.flags.writeable = False

        results = hands.process(image) # Make detection

        # image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

            HandLandmark = results.multi_hand_landmarks
            hand_row = list (np.array([[i.x, i.y, i.z] for i in hand_landmarks.landmark]).flatten())
            X = pd.DataFrame([hand_row])
            hand_language_class = model.predict(X)[0]
            hand_language_prob = model.predict_proba(X)[0]
#             print(hand_language_class, hand_language_prob)
       
            text = str(hand_language_class)
            textsize = cv2.getTextSize(text, font, 4, 2)[0]
            # get coords based on boundary
            textX = 0
            textY = textsize[1]
            cv2.putText(image, text, (textX, textY),
                    font, 4, (255, 255, 255), 2, cv2.LINE_AA)
            
        cv2.imshow(window_name, image)
        
        if ret == False or cv2.waitKey(10) == ord("q"):
            break
            
    cap.release()
    cv2.destroyAllWindows()
